In [1]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive, prove

In [2]:
# The smart contract as a transition system.
ts = Ts('auction')

addrSort = z3.BitVecSort(16)

auctionEndTime = z3.Int('auctionEndTime')

ended, endedOut = ts.add_var(z3.BoolSort(), name='ended')

pendingReturns, pendingReturnsOut = ts.add_var(z3.ArraySort(addrSort, z3.IntSort()),
        name= 'pendingReturns')

highestBidder, highestBidderOut = ts.add_var(addrSort, name='highestBidder')

highestBid, highestBidOut = ts.add_var(z3.IntSort())

withdrawCounts, withdrawCountsOut = ts.add_var(z3.ArraySort(addrSort, z3.IntSort()), 
        name='withdrawCount')

In [3]:
p = z3.BitVec('p',16)
ts.Init = z3.And(z3.ForAll(p,pendingReturns[p]==0),
        highestBid == 0,
        highestBidder == 0,
        ended == False,
        z3.ForAll(p, withdrawCounts[p]==0)
)

In [4]:
now = z3.Int('now')
value = z3.Int('value')
sender = z3.BitVec('sender', 16)

tr_bid = z3.And(now < auctionEndTime,
        value > highestBid,
        highestBid > 0,
        z3.Not(ended),
        z3.Or(
            z3.And(highestBid != 0, 
                pendingReturnsOut == z3.Update(pendingReturns,highestBidder,
                                        pendingReturns[highestBidder]+highestBid)),
            z3.And(highestBid == 0, pendingReturnsOut==pendingReturns)
        ),
        highestBidOut == value,
        highestBidderOut == sender,
        withdrawCountsOut == withdrawCounts,
        endedOut == ended,
        )

tr_withdraw = z3.And(
        pendingReturns[sender]>0,
        ended,
        pendingReturnsOut == z3.Update(pendingReturns, sender, 0),
        withdrawCountsOut == z3.Update(withdrawCounts, sender, withdrawCounts[sender]+1),
        endedOut == ended,
        highestBidderOut==highestBidder,
        highestBidOut == highestBid
)

tr_end = z3.And(
        now >= auctionEndTime,
        z3.Not(ended),
        endedOut == True,
        pendingReturnsOut == pendingReturns,
        withdrawCountsOut==withdrawCounts,
        highestBidderOut==highestBidder,
        highestBidOut == highestBid
)

ts.Tr = z3.Or( 
    tr_bid, 
    tr_withdraw, 
    tr_end
    )

In [5]:
pp = z3.Const('pp', addrSort)
ts_property = z3.ForAll(pp, withdrawCounts[pp] <= 1)

In [12]:
pp = z3.Const('pp', addrSort)
# p0 = z3.And(z3.And(withdrawCounts[pp]!=0, withdrawCounts[pp]>=0))
p0 = withdrawCounts[pp]!=0
p1 = z3.ForAll(pp, z3.Implies(p0, z3.Not(pendingReturns[pp]>0)))
p2 = z3.ForAll(pp, z3.Implies(p0, ended))
lemma = z3.And(p1,p2)
prove_inductive(ts,lemma)

Prove init => property.
proved
Prove property is inductive.
proved


In [13]:
prove_inductive(ts, ts_property, lemma)

Prove init => property.
proved
Prove property is inductive.
proved


In [14]:
prove_inductive(ts,p1)

Prove init => property.
proved
Prove property is inductive.
failed to prove:  unknown
Implies(And(ForAll(pp,
                   Implies(And(And(withdrawCount[pp] != 0,
                                   withdrawCount[pp] >= 0)),
                           pendingReturns[pp] == 0)),
            Or(And(now < auctionEndTime,
                   value > v_3,
                   v_3 > 0,
                   Not(ended),
                   Or(And(v_3 != 0,
                          pendingReturns' ==
                          Store(pendingReturns,
                                highestBidder,
                                pendingReturns[highestBidder] +
                                v_3)),
                      And(v_3 == 0,
                          pendingReturns' == pendingReturns)),
                   v_out_3 == value,
                   highestBidder' == sender,
                   withdrawCount' == withdrawCount,
                   ended' == ended),
               And(pendingReturns[se

Z3Exception: model is not available

In [17]:
pp = z3.Const('pp', addrSort)

p1 = z3.ForAll(pp, z3.Implies(pendingReturns[pp] > 0, withdrawCounts[pp] == 0))
p2 = z3.ForAll(pp, z3.Implies(z3.Not(ended), withdrawCounts[pp] == 0))


In [26]:
lemma = z3.And(p1,p2)
prove_inductive(ts,lemma)

Prove init => property.
proved
Prove property is inductive.
proved


In [13]:
prove_inductive(ts, ts_property , lemma)

Prove init => property.
proved
Prove property is inductive.
proved


In [6]:
pp = z3.Const('pp', addrSort)
p1 = z3.ForAll(pp, z3.Implies(withdrawCounts[pp]>0, pendingReturns[pp]==0))
p2 = z3.ForAll(pp, z3.Implies(withdrawCounts[pp]>0,  ended))
lemma = z3.And(p1,p2)

In [7]:
prove_inductive(ts, lemma)

Prove init => property.
proved
Prove property is inductive.
proved


In [8]:
prove_inductive(ts,ts_property,lemma)

Prove init => property.
proved
Prove property is inductive.
proved
